# player

> This module training optimizing of RL agents on the electricity market environment.

In [ ]:
# | default_exp player

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import pickle
from abc import ABC
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import optuna
import torch
import yaml
from sb3_contrib import MaskablePPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.monitor import Monitor
from tqdm.notebook import tqdm

from electricity_market.env import ElectricityMarketEnv, EnvConfig
from electricity_market.utils import EvaluationData, TrainingData

In [ ]:
# | export
N_EPISODES = 3
N_TRAILS = 10
SEEDS = [
    111111,
    121212,
    123456,
    200000,
    217890,
    222222,
    224775,
    234567,
    253084,
    285234,
    312135,
    314831,
    333333,
    345678,
    406339,
    444444,
    471678,
    555555,
    562845,
    666666,
    701753,
    755460,
    761386,
    777777,
    789391,
    888888,
    993068,
    979797,
    987654,
    999999,
]
ENV_CONFIG = EnvConfig()

QUICK_MODE = True

if QUICK_MODE:
    ENV_CONFIG = EnvConfig(max_timestep=10)
    SEEDS = [10000]

training_data_per_agent = {}
evaluation_data_per_agent = {}

In [ ]:
# | export


@dataclass
class TrainingData:
    steps: list[int]
    episodes: list[int]
    rewards: list[float]


@dataclass
class EvaluationData:
    episodes: list[int]
    rewards: list[float]

In [ ]:
# | export


class Agent(ABC):
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        """
        Evaluate the model, and return EvaluationData.
        """
        raise NotImplementedError


class ModelAgent(Agent):
    def __init__(self, name, env, model, device):
        super().__init__(name)
        self.device = device
        self.model = model
        self.env = env

    def train(self) -> TrainingData:
        """
        Train the model, and return TrainingData.
        """
        all_rewards = []
        all_steps = []
        total_steps = 0
        checkpoint_callback = CheckpointCallback(save_freq=1000, save_path="./logs/")

        # Training loop
        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Training episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                steps = 0
                done = False

                while not done:
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)
                    if isinstance(self, MaskableAgent):

                        action, _ = self.model.predict(
                            obs_tensor, action_masks=MaskableAgent.mask_fn(self.env)
                        )
                    else:
                        action, _ = self.model.predict(obs_tensor)

                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)
                    steps += 1

                    if done or truncated:

                        self.model.learn(
                            total_timesteps=steps, callback=checkpoint_callback
                        )

                all_rewards.append(np.sum(episode_rewards))
                total_steps += steps
                all_steps.append(total_steps)

        return TrainingData(
            steps=all_steps,
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluate the model, and return EvaluationData.
        """
        all_rewards = []

        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Evaluation episodes"):
                obs, _ = self.env.reset(seed=seed)
                episode_rewards = []
                done = False

                while not done:
                    # Convert observation to torch tensor
                    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(self.device)

                    if isinstance(self, MaskableAgent):
                        action, _ = self.model.predict(
                            obs_tensor,
                            action_masks=MaskableAgent.mask_fn(self.env),
                            deterministic=True,
                        )
                    else:
                        action, _ = self.model.predict(obs_tensor, deterministic=True)
                    obs, reward, done, truncated, _ = self.env.step(action)
                    episode_rewards.append(reward)

                    if render:
                        self.env.render()

                    if truncated:
                        break

                all_rewards.append(np.sum(episode_rewards))

        return EvaluationData(
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

    def save_model(self, model_path: Path) -> None:
        self.model.save(str(model_path))

    def load_model(self, model_path: Path) -> None:
        self.model.load(str(model_path))


class MaskableAgent(Agent):
    @staticmethod
    def mask_fn(env):
        """
        Placeholder mask function if needed.
        """
        if isinstance(env, Monitor):
            return env.env.action_masks()
        return env.action_masks()

In [ ]:
# | export


class MaskableRandomAgent(MaskableAgent):
    def __init__(
        self,
        env_config: EnvConfig | None = None,
        render_mode: str | None = None,
        name: str = "MaskableRandomAgent",
    ):
        """
        Initialize the agent and create the environment.
        """
        super().__init__(name=name)
        self.env = ActionMasker(
            ElectricityMarketEnv(env_config, render_mode=render_mode), self.mask_fn
        )

    def evaluate(self, render: bool = False) -> EvaluationData:
        """
        Evaluates the random agent by executing episodes in the environment.
        """
        all_rewards = []

        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="episodes", leave=False):
                episode_rewards = []

                obs, _ = self.env.reset(seed=seed)
                done = False
                total_reward = 0.0

                while not done:
                    action_mask = self.env.action_masks()
                    valid_actions = np.where(action_mask)[0]  # Get valid actions
                    action = np.random.choice(
                        valid_actions
                    )  # Select random valid action

                    obs, reward, done, truncated, _ = self.env.step(action)
                    total_reward += reward

                    if render:
                        self.env.render()

                    if truncated:
                        break

                episode_rewards.append(total_reward)

                # Append episode results to the total list
                all_rewards.append(total_reward)

        return EvaluationData(
            episodes=list(range(len(all_rewards))),
            rewards=all_rewards,
        )

In [ ]:
# | export


class A2CAgent(ModelAgent):
    """A2C Agent for the Electricity Market Environment."""

    def __init__(
        self,
        env_config: EnvConfig | None = None,
        render_mode: str | None = None,
        name: str = "A2CAgent",
    ):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        env = Monitor(
            ElectricityMarketEnv(env_config, render_mode=render_mode),
        )
        model = A2C(
            "MlpPolicy",
            env,
            verbose=0,
            tensorboard_log=f"./{name}_tensorboard/",
            device=device,
        )
        super().__init__(name=name, env=env, model=model, device=device)

In [ ]:
# | export


class MaskablePPOAgent(ModelAgent, MaskableAgent):
    """Maskable PPO Agent for the Electricity Market Environment."""

    def __init__(
        self,
        env_config: EnvConfig | None = None,
        render_mode: str | None = None,
        name: str = "MaskablePPOAgent",
    ):
        env = Monitor(
            ActionMasker(
                ElectricityMarketEnv(env_config, render_mode=render_mode),
                self.mask_fn,
            )
        )
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = MaskablePPO(
            MaskableActorCriticPolicy,
            env,
            verbose=0,
            tensorboard_log=f"./{name}_tensorboard/",
            device=device,
        )
        super().__init__(name=name, env=env, model=model, device=device)
        self.optimized_hyperparameters = {}
        self.env_config = env_config or EnvConfig()

    @staticmethod
    def collect_episodes_rewards(model, env) -> list[float]:
        """
        Collect rewards from evaluating the agent for a given number of episodes.
        """
        rewards = []
        for seed in tqdm(SEEDS, desc="seeds"):
            for _ in tqdm(range(N_EPISODES), desc="Collect episodes"):
                obs, _ = env.reset(seed=seed)
                episode_rewards = 0
                done = False

                while not done:
                    action, _ = model.predict(
                        obs, deterministic=True, action_masks=MaskableAgent.mask_fn(env)
                    )
                    obs, reward, done, truncated, _ = env.step(action)
                    episode_rewards += reward
                rewards.append(episode_rewards)

        return rewards

    def optimize(self) -> None:
        """
        Optimize the agent with hyperparameters and return TrainingData.
        """

        def objective(trial):
            # Hyperparameter search space
            learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
            n_steps = trial.suggest_int("n_steps", 32, 1024, log=True)
            batch_size = trial.suggest_int("batch_size", 16, 256, log=True)
            gamma = trial.suggest_float("gamma", 0.9, 0.9999)
            gae_lambda = trial.suggest_float("gae_lambda", 0.8, 1.0)
            ent_coef = trial.suggest_float("ent_coef", 0.0, 0.02)
            vf_coef = trial.suggest_float("vf_coef", 0.1, 1.0)
            clip_range = trial.suggest_float("clip_range", 0.1, 0.3)
            max_grad_norm = trial.suggest_float("max_grad_norm", 0.1, 1.0)

            # Set up environment and model with the sampled hyperparameters
            env = Monitor(
                ActionMasker(
                    ElectricityMarketEnv(self.env_config, render_mode="human"),
                    self.mask_fn,
                )
            )

            model = MaskablePPO(
                MaskableActorCriticPolicy,
                env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                ent_coef=ent_coef,
                vf_coef=vf_coef,
                clip_range=clip_range,
                max_grad_norm=max_grad_norm,
                verbose=0,
                tensorboard_log=f"./{self.name}_tensorboard/",
                device=self.device,
            )

            model.learn(
                total_timesteps=self.env_config.max_timestep,
                use_masking=True,
                reset_num_timesteps=False,
            )

            # Collect rewards for evaluation
            episode_rewards = self.collect_episodes_rewards(model, env)

            # Return the mean reward as the optimization goal
            return np.mean(episode_rewards)

        # Create a study to optimize the objective function
        study = optuna.create_study(
            study_name=self.name,
            storage="sqlite:///optuna_study.db",
            load_if_exists=True,
            direction="maximize",
            pruner=optuna.pruners.HyperbandPruner(),
            sampler=optuna.samplers.TPESampler(),
        )

        # Optimize using the defined objective function
        study.optimize(objective, n_trials=N_TRAILS, n_jobs=-1, show_progress_bar=True)

        # Save the best learned_hyperparameters
        self.optimized_hyperparameters = study.best_params

        # Re-train the model with the optimized learned_hyperparameters
        self.model = MaskablePPO(
            MaskableActorCriticPolicy,
            self.env,
            **self.optimized_hyperparameters,
            verbose=0,
            tensorboard_log=f"./{self.name}_tensorboard/",
            device=self.device,
        )

    def export_hyperparameters(self, filename: str):
        """
        Export optimized learned_hyperparameters to a YAML file.
        """
        with open(filename, "w") as file:
            yaml.dump(self.optimized_hyperparameters, file)

### Evaluation MaskableRandom on ElectricityMarketEnv


In [ ]:
# | hide
maskable_random_agent = MaskableRandomAgent(render_mode="human", env_config=ENV_CONFIG)

evaluation_data_per_agent[maskable_random_agent.name] = maskable_random_agent.evaluate()

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation A2C on ElectricityMarketEnv


In [ ]:
# | hide
a2c_agent = A2CAgent(render_mode="human", env_config=ENV_CONFIG)

training_data_per_agent[a2c_agent.name] = a2c_agent.train()

a2c_agent.save_model(f"{a2c_agent.name}.model")

evaluation_data_per_agent[a2c_agent.name] = a2c_agent.evaluate()

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation MaskablePPO with default hyperparameters on ElectricityMarketEnv

In [ ]:
# | hide
maskable_ppo_agent = MaskablePPOAgent(render_mode="human", env_config=ENV_CONFIG)

training_data_per_agent[maskable_ppo_agent.name] = maskable_ppo_agent.train()

maskable_ppo_agent.save_model(f"{maskable_ppo_agent.name}.model")

evaluation_data_per_agent[maskable_ppo_agent.name] = maskable_ppo_agent.evaluate()

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation MaskablePPO with optimized hyperparameters on ElectricityMarketEnv

In [ ]:
# | hide
optimized_maskable_ppo_agent = MaskablePPOAgent(
    render_mode="human", env_config=ENV_CONFIG, name="OptimizedMaskablePPOAgent"
)
optimized_maskable_ppo_agent.optimize()

optimized_maskable_ppo_agent.export_hyperparameters(
    f"{optimized_maskable_ppo_agent.name}.yaml"
)


training_data_per_agent[optimized_maskable_ppo_agent.name] = (
    optimized_maskable_ppo_agent.train()
)

optimized_maskable_ppo_agent.save_model(f"{optimized_maskable_ppo_agent.name}.model")

evaluation_data_per_agent[optimized_maskable_ppo_agent.name] = (
    optimized_maskable_ppo_agent.evaluate()
)

[I 2025-02-25 12:38:55,670] A new study created in RDB with name: OptimizedMaskablePPOAgent


  0%|          | 0/10 [00:00<?, ?it/s]

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

Collect episodes:   0%|          | 0/3 [00:00<?, ?it/s]

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

...

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Adding expert knowledge to the masking function making learning more efficient

In [ ]:
# | export


def is_action_safe(self, action: int) -> bool:
    charge_amount = self._charge_amount(action)
    target_state_of_charge = self._current_state_of_charge + charge_amount
    low, high = self._battery_safe_range
    return high > target_state_of_charge > low


def expert_knowledge_action_masks(self) -> np.ndarray:
    mask = np.array(
        [
            self._is_action_valid(action) and self.is_action_safe(action)
            for action in range(self.action_space.n)
        ],
        dtype=bool,
    )
    if not np.any(mask):  # If all actions are invalid, force one to be valid
        mask[len(mask) // 2] = True
    return mask

In [ ]:
# | hide
# Dynamically overriding action_masks to ElectricityMarketEnv
setattr(ElectricityMarketEnv, "action_masks", expert_knowledge_action_masks)
# Dynamically overriding injection is_action_safe to ElectricityMarketEnv
setattr(ElectricityMarketEnv, "is_action_safe", is_action_safe)



### Evaluation MaskableRandomAgent with Expert Knowledge on ElectricityMarketEnv

In [ ]:
# | hide
expert_maskable_random_agent = MaskableRandomAgent(
    render_mode="human", env_config=ENV_CONFIG, name="ExpertMaskableRandomAgent"
)

evaluation_data_per_agent[expert_maskable_random_agent.name] = (
    expert_maskable_random_agent.evaluate()
)

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation MaskablePPO with default hyperparameters and Expert Knowledge on ElectricityMarketEnv

In [ ]:
# | hide
expert_maskable_ppo_agent = MaskablePPOAgent(
    render_mode="human", env_config=ENV_CONFIG, name="ExpertMaskablePPOAgent"
)

training_data_per_agent[expert_maskable_ppo_agent.name] = (
    expert_maskable_ppo_agent.train()
)

expert_maskable_ppo_agent.save_model(f"{expert_maskable_ppo_agent.name}.model")

evaluation_data_per_agent[expert_maskable_ppo_agent.name] = (
    expert_maskable_ppo_agent.evaluate()
)

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Training episodes:   0%|          | 0/3 [00:00<?, ?it/s]

...

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

### Evaluation MaskablePPO with optimized hyperparameters and Expert Knowledge on ElectricityMarketEnv


In [ ]:
# | hide
optimized_expert_maskable_ppo_agent = MaskablePPOAgent(
    render_mode="human", env_config=ENV_CONFIG, name="OptimizedExpertMaskablePPOAgent"
)
optimized_expert_maskable_ppo_agent.optimize()

optimized_expert_maskable_ppo_agent.export_hyperparameters(
    f"{optimized_expert_maskable_ppo_agent.name}.yaml"
)

training_data_per_agent[optimized_expert_maskable_ppo_agent.name] = (
    optimized_expert_maskable_ppo_agent.train()
)

optimized_expert_maskable_ppo_agent.save_model(
    f"{optimized_expert_maskable_ppo_agent.name}.model"
)

evaluation_data_per_agent[optimized_expert_maskable_ppo_agent.name] = (
    optimized_expert_maskable_ppo_agent.evaluate()
)

[I 2025-02-25 14:46:00,405] A new study created in RDB with name: OptimizedExpertMaskablePPOAgent


  0%|          | 0/10 [00:00<?, ?it/s]

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

Collect episodes:   0%|          | 0/3 [00:00<?, ?it/s]

seeds:   0%|          | 0/30 [00:00<?, ?it/s]

...

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation episodes:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# | hide

with open("training_data_per_agent.pkl", "wb") as f:
    pickle.dump(training_data_per_agent, f)

with open("evaluation_data_per_agent.pkl", "wb") as f:
    pickle.dump(evaluation_data_per_agent, f)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()